In [360]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Assortment Slide Duplicate\Assortment Replacement Function New.ipynb"

In [361]:
ManufOrTopC ="Top Companies" #Manufacturer or Top Companies
BrandOrTopB= "Top Brands" #Brand or Top Brands
prodORitem = "Business Name"
prodORitemfields= "SKU"

client_manuf =["Malard Nicolas"]
client_brands = ["Nicolas"]

decimals = 2
sign = "after"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

categories =["Effervescents"]
sectors = ["Champagnes","Mousseux","Effervescents Sans Alcool"]
segments = ["Blanc De Blancs","Bruts","Crus","Extra Brut","Millesimes","Rosés","Mousseux Bourgogne","Mousseux Italiens","Mousseux Val De Loire","Mousseux Alsace","Mousseux Bordelais"]
subsegments= []
subcategories= []

national = False
customareas= "REVENUE"
areas = ["CHANNEL","REGIONS", f'{customareas}']
 
regions_RET  = []
channels_RET = []
market_RET = []
 
regions_CHAN = ["NICOLAS"]
channels_CHAN = ["NICOLAS QCN","NICOLAS VCN","NICOLAS QCT","NICOLAS QCA","NICOLAS CCP"]
market_CHAN = []

regions_REG = []
channels_REG = ["NICOLAS IDF","NICOLAS PAC","NICOLAS RHO","NICOLAS AQU","NICOLAS EST"]
market_REG = []

regions_CUST = []
channels_CUST = ["CA A","CA B","CA C","CA D","CA E"]
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending June 2025"
years = ['2023', '2024','2025']
end_date = "2025-07-01"
unit = "‘000000s"
unitDeviation = 1000000
OpenEditData=True
RunManuf= False
Runwithoutclientorder= False

client_brands_competitor = client_brands 
valueToReplace = {}


In [362]:
loaded_data = {}
datasets_path = os.getcwd()+"/Assortment Datasets NewEX/"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

In [363]:
def mixAssortmentCleaning(assortment, cumulativeShare,slideby="Top Brands",lis=""):
    # Initialize dictionaries to store modified data
    cumulativeShareModifiedBrand = {}
    assortmentModified = {}
    assortmentModifiedBrand = {}
    assortmentModifiedTotal = {}
    assortmentClient = {}

    # Process cumulative share data
    for key, value in cumulativeShare.items():
        dfcumulative=cumulativeShare[key].copy()
        # dfcumulative=DetectHeader(dfcumulative)
        if dfcumulative.shape[0] != 0:  # If the dataframe is not empty
            newKey = key
            cumulativeShareModifiedBrand[newKey] = dfcumulative.replace(np.nan, 0)  # Replace NaNs with 0 and store

    # Process assortment data
    for key, value in assortment.items():
        df=assortment[key].copy()

        # df=DetectHeader(df)
        df[slideby] = df[slideby].ffill()  # Forward fill 'Top Brands' column

        # Replace specific values in 'Top Brands' as per 'valueToReplace' dictionary
        for val, replacer in valueToReplace.items():
            df[slideby] = df[slideby].str.replace(val, replacer)
        
        dfBrand = df[~df[slideby].str.contains('Total')]  # Filter out rows containing 'Total' in 'Top Brands'
        dfTotal = df[df[slideby].str.contains('Total') & (df[slideby] != 'Grand Total')].reset_index(drop=True)
        dfTotal[slideby] = dfTotal[slideby].str.replace(' Total', '')  # Adjust 'Top Brands' column for total rows

        if df.shape[0] != 0:  # If the dataframe is not empty
            newKey = key

            # Process client-specific data for each brand
            for brand in lis:

                if df[df[slideby] == brand].shape[0] > 0:  # Check if brand data exists in dataframe

                    #assortmentClient[newKey + ' | ' + brand] = df[df['Top Brands'] == brand].replace(np.nan, 0)
                    brand_df = df[df[slideby] == brand].replace(np.nan, 0) 
                    brand_df = brand_df.sort_values(by='Value Share', ascending = False)
                    brand_df['Value Share Rescale'] = brand_df['Value Share'] / sum(brand_df['Value Share'])
                    brand_df['Cumulative Share'] = brand_df['Value Share Rescale'].cumsum()
                    brand_df = brand_df[brand_df["WD"] != 0]
                    #brand_df['Final Cumlative Share'] = 100 - brand_df['Cumlative Share']*100
                    assortmentClient[newKey + ' | ' + brand] = brand_df.merge(
                        cumulativeShareModifiedBrand[newKey], how='left', on= prodORitem
                    )
                    #assortmentClient[newKey + ' | ' + brand] = assortmentClient[newKey + ' | ' + brand].merge(
                    #    cumulativeShareModifiedBrand[newKey], how='left', on= prodORitem
                    #)            
            # Store modified data for total, brand, and overall assortment
            assortmentModified[newKey] = df.replace(np.nan, 0)
            assortmentModifiedBrand[newKey] = dfBrand.replace(np.nan, 0)
            assortmentModifiedBrand[newKey] = assortmentModifiedBrand[newKey][assortmentModifiedBrand[newKey]["WD"] != 0]
            assortmentModifiedBrand[newKey] = assortmentModifiedBrand[newKey].merge(
                cumulativeShareModifiedBrand[newKey], how='left', on=prodORitem
            )
            
            assortmentModifiedTotal[newKey] = dfTotal.replace(np.nan, 0)
    
    return assortmentModifiedBrand, assortmentModifiedTotal, assortmentClient


In [364]:
assortmentBrand,assortmentTotal,assortmentClient=mixAssortmentCleaning(assortment,cumulative_share,slideby=f'{BrandOrTopB}',lis=client_brands_competitor)
dropemptydf(assortmentBrand)
assortmentmanuf,assortmentTotalmanuf,assortmentClientmanuf=mixAssortmentCleaning(assortment_manuf,cumulative_share,slideby=f'{ManufOrTopC}',lis=client_manuf)
dropemptydf(assortmentmanuf)

In [365]:
assortmentClientManufAppend= assortmentClientmanuf
assortmentClientManufAppend.update(assortmentClient)
dropemptydf(assortmentClientManufAppend)

In [366]:
"""
Cleans and processes assortment for brand-specific analysis.

Args:
assortmentbybrand (dict): Dictionary containing assortment data by brand.

Returns:
tuple: 
   - assortmentModifiedBrand (dict): Dictionary containing cleaned and modified assortment data by brand for slide 1
"""

# Initialize dictionaries to store modified data
assortmentbybrandfinal = {}
# Process assortment data
for key, value in assortmentbybrand.items():
   df=assortmentbybrand[key].copy()
   # df=DetectHeader(df)
   if BrandOrTopB != "Top Brands":
      df= df.rename(columns={"Brand":"Top Brands"})
   # Replace specific values in 'Top Brands' as per 'valueToReplace' dictionary
   for val, replacer in valueToReplace.items():
      df['Top Brands'] = df['Top Brands'].str.replace(val, replacer)
   
   dfBrand = df[df['Top Brands'] != 'Grand Total'].reset_index(drop=True)
   if dfBrand.shape[0] != 0:  # If the dataframe is not empty
      assortmentbybrandfinal[key] = dfBrand.replace(np.nan, 0)

In [367]:
"""
Cleans and processes assortment for brand-specific analysis.

Args:
assortmentbybrand (dict): Dictionary containing assortment data by brand.

Returns:
tuple: 
   - assortmentModifiedBrand (dict): Dictionary containing cleaned and modified assortment data by brand for slide 1
"""

# Initialize dictionaries to store modified data
assortmentbymanufFinal = {}
# Process assortment data
for key, value in assortmentbymanuf.items():
   df=assortmentbymanuf[key].copy()
   # df=DetectHeader(df)
   dfManuf = df[df[ManufOrTopC] != 'Grand Total'].reset_index(drop=True)
   dfManuf = dfManuf[dfManuf[ManufOrTopC] != 'All Others'].reset_index(drop=True)
   if dfManuf.shape[0] != 0:  # If the dataframe is not empty
         assortmentbymanufFinal[key] = dfManuf.replace(np.nan, 0)

In [368]:
assortmentBrandSorted= dfSort(assortmentbybrandfinal, client_brands, 'Top Brands', num=10,salesCol='Value Share')


In [369]:
assortmentManufSorted= dfSort(assortmentbymanufFinal, [], ManufOrTopC, num=10,salesCol='Value Share')


In [370]:
assortmentBrandNOTSorted= dfSort(assortmentbybrandfinal, [], 'Top Brands', num=10,salesCol='Value Share')

In [371]:
assortmentTotalSorted= dfSort(assortmentTotal, client_brands, 'Top Brands', num=10,salesCol='Value Share')

### Slide 3,4 cumulative share Cleaning

In [372]:
def cumulativeShareCleaningTopPercent(assortment, cumulativeShare, percent=0.5):
    """
    Cleans and processes assortment and cumulative share data, filtering out the top percentage of products.
    
    Args:
    assortment (dict): Dictionary containing assortment data.
    cumulativeShare (dict): Dictionary containing cumulative share data.
    percent (float): The top cumulative product share percentage threshold to filter by (default is 0.5).
    
    Returns:
    dict: A dictionary containing the cleaned and processed data for the top percentage of products.
    """
    
    # Initialize dictionaries to store modified data
    assortmentModified = {}
    assortmentModifiedBrand = {}
    topPercent = {}
    cumulativeShareModifiedBrand = {}

    # Process cumulative share data
    for key, value in cumulativeShare.items():
        dfcumulative=cumulativeShare[key].copy()
        # dfcumulative=DetectHeader(dfcumulative)
        if dfcumulative.shape[0] != 0:  # If the dataframe is not empty
            newKey = key
            cumulativeShareModifiedBrand[newKey] = dfcumulative.replace(np.nan, 0)  # Replace NaNs with 0 and store

    # Process assortment data
    for key, value in assortment.items():
        df=assortment[key].copy()
        # df=DetectHeader(df)
        if BrandOrTopB != "Top Brands":
            df= df.rename(columns={"Brand":"Top Brands"})
        df['Top Brands'] = df['Top Brands'].ffill()  # Forward fill 'Top Brands' column
        
        # Replace specific values in 'Top Brands' as per 'valueToReplace' dictionary
        for val, replacer in valueToReplace.items():
            df['Top Brands'] = df['Top Brands'].str.replace(val, replacer)
        
        dfBrand = df[~df['Top Brands'].str.contains('Total')]  # Filter out rows containing 'Total' in 'Top Brands'
        
        if df.shape[0] != 0:  # If the dataframe is not empty
            newKey = key

            # Store modified data for overall and brand-specific assortment
            assortmentModified[newKey] = df.replace(np.nan, 0)
            assortmentModifiedBrand[newKey] = dfBrand.replace(np.nan, 0)
            assortmentModifiedBrand[newKey] = assortmentModifiedBrand[newKey].merge(
                cumulativeShareModifiedBrand[newKey], how='left', on=prodORitem
            )
            
            # Filter and store data for products within the specified cumulative share percentage
            # if (assortmentModifiedBrand[newKey][assortmentModifiedBrand[newKey]['Cumulative '+prodORitem+' Share'] <= percent].shape[0] > 0):
            
            topPercent[newKey] = assortmentModifiedBrand[newKey]

    return topPercent


In [373]:
# cumulativeShareTop50 = cumulativeShareCleaningTopPercent(assortment,cumulative_share, percent =0.5)

In [374]:
cumulativeShareAll = cumulativeShareCleaningTopPercent(assortment,cumulative_share, percent =100)

### Slide 6,7 SKU Productivity Analysis Cleaning

In [375]:
def productivityAnalysisCleaning(assortmentModifiedBrand):
    """
    Cleans and filters the assortment data for client brands based on specific criteria.
    
    Args:
    assortmentModifiedBrand (dict): Dictionary containing modified assortment data by brand.
    
    Returns:
    dict: A dictionary containing the cleaned and filtered data for each client brand.
    """
    
    assortmentClientBrand = {}  # Initialize a dictionary to store the cleaned data for client brands
    
    # Iterate over the modified assortment data
    for key, value in assortmentModifiedBrand.items():
        # Filter the dataframe for client brands with Net Sales >= 1000 and WD not equal to 0
        df = value[(value['Top Brands'].isin(client_brands)) & (value['Net Sales'] >= 1000)]
        df = df[df['WD'] != 0]
        
        # If the filtered dataframe is not empty
        if df.shape[0] != 0:
            # Iterate over each client brand
            for brand in client_brands:
                # If the brand exists in the filtered dataframe
                if df[df['Top Brands'] == brand].shape[0] > 0:
                    # Store the filtered data in the dictionary with a combined key of original key and brand name
                    assortmentClientBrand[key + ' | ' + brand] = df[df['Top Brands'] == brand]
    
    return assortmentClientBrand


In [376]:
assortmentClientBrand=productivityAnalysisCleaning(assortmentBrand)


## New Slide 

In [377]:
def clean_brand(brands):
    new_brands ={}
    for key, df in brands.items():
        if df.shape[0] != 0:  
            new_brands[key] =df.replace(np.nan, -1)  
    return new_brands


In [378]:
def clean_manuf(manuf):
    new_manuf ={}
    for key, df in manuf.items():
        if df.shape[0] != 0:  
            # newKey = key
            # if key.split(' | ')[0] not in categories:  
            #     newKey = key.split(' | ')[1] + ' | ' + key.split(' | ')[0]
            new_manuf[key] = df.replace(np.nan, -1)  
    return new_manuf


In [379]:
def clean_total(total):
    new_total ={}
    for key, df in total.items():
        # new_df = DetectHeader(df)
        if df.shape[0] != 0:
            new_df = df.drop(['SKU Share', 'Value Share'], axis=1)
            new_total[key] = new_df.replace(np.nan, -1) 
    return new_total

In [380]:
def merge_total_brand(total, brand, column,scope):
    brand_total ={}
    for key in brand.keys():
        market = key.split(' | ')[1]
        for small_key in total.keys():
            if market == small_key:
                # print(market, small_key, key)
                df = pd.merge(total[small_key], brand[key], how = 'left',on=column,suffixes=('_total', '_brand')).replace(np.nan,0)
                print(df.columns)
                df['Fair Share'] = df.apply(lambda x: ((x['Av No SKUs_brand'] / x['SKU Share']) * x['Value Share'] if x['SKU Share'] != 0 else 0), axis=1)
                df ['Listing Opportunity'] = df['Fair Share'].astype(float).round(0) - df['Av No SKUs_brand'].astype(float).round(0)
                grand_row = df[df[column] == 'Grand Total']
                other_rows = df[df[column].isin(scope)]
                df = pd.concat([grand_row, other_rows], ignore_index=True)
                if df.shape[0] > 0:
                    brand_total[key] = df

    return brand_total

In [382]:
if len(sectors) > 0:
    Sector_brands = clean_brand(Sector_dfs_brands)
    Sector_manuf = clean_manuf(Sector_dfs_manuf)
    Sector_total = clean_total(Sector_dfs_total)
    Sector_brand_total = merge_total_brand(Sector_total, Sector_brands, 'Sector',sectors)
    Sector_manuf_total = merge_total_brand(Sector_total, Sector_manuf, 'Sector',sectors)
    final_Sector = Sector_manuf_total | Sector_brand_total


if len(segments) > 0:
    Segment_brands = clean_brand(Segment_dfs_brands)
    Segment_manuf = clean_manuf(Segment_dfs_manuf)
    Segment_total = clean_total(Segment_dfs_total)
    Segment_brand_total = merge_total_brand(Segment_total, Segment_brands, 'Segment',segments)
    Segment_manuf_total = merge_total_brand(Segment_total, Segment_manuf, 'Segment',segments)
    final_Segment = Segment_manuf_total | Segment_brand_total


if len(subcategories) > 0:
    SubCategory_brands = clean_brand(SubCategory_dfs_brands)
    SubCategory_manuf = clean_manuf(SubCategory_dfs_manuf)
    SubCategory_total = clean_total(SubCategory_dfs_total)
    SubCategory_brand_total = merge_total_brand(SubCategory_total, SubCategory_brands, 'SubCategory',subcategories)
    SubCategory_manuf_total = merge_total_brand(SubCategory_total, SubCategory_manuf, 'SubCategory',subcategories)
    final_SubCategory = SubCategory_manuf_total | SubCategory_brand_total


if len(subsegments) > 0:
    Subsegment_brands = clean_brand(SubSegment_dfs_brands)
    Subsegment_manuf = clean_manuf(SubSegment_dfs_manuf)
    Subsegment_total = clean_total(SubSegment_dfs_total)
    Subsegment_brand_total = merge_total_brand(Subsegment_total, Subsegment_brands,'SubSegment',subsegments)
    Subsegment_manuf_total = merge_total_brand(Subsegment_total, Subsegment_manuf,'SubSegment',subsegments)
    final_Subsegment = Subsegment_manuf_total | Subsegment_brand_total




Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Value Share'],
      dtype='object')
Index(['Sector', 'Av No SKUs_total', 'Av No SKUs_brand', 'SKU Share',
       'Va

In [ ]:
index = [0,0 if RunManuf else None,1,3,4,5 if len(client_brands)>0 else None,6 if len(client_brands)>0 else None, *[7]*((1 if sectors else 0) + (1 if segments else 0) + (1 if subsegments else 0) + (1 if subcategories else 0)),0 if Runwithoutclientorder else None]
        #  2 if len(client_brands)>0 else None ##TOP%CUM
index = [x for x in index if x is not None]

duplication = [len(assortmentBrandSorted.keys()),len(assortmentManufSorted.keys()) if RunManuf else None,len(assortmentBrand.keys()),len(assortmentClientManufAppend.keys()) if len(client_brands) >0 else None,len(assortmentBrand.keys()) , len(assortmentClientBrand.keys()) if len(client_brands)>0 else None , len(assortmentClientBrand.keys()) if len(client_brands)>0 else None,
            #    len(cumulativeShareTop50.keys()), ##TOP%CUM
               len(final_Sector.keys()) if len(sectors) >0 else None, len(final_Segment.keys()) if len(segments) > 0 else None, len(final_SubCategory.keys()) if len(subcategories) >0 else None, len(final_Subsegment.keys()) if len(subsegments) > 0 else None ,len(assortmentBrandNOTSorted.keys()) if Runwithoutclientorder else None]
duplication = [x for x in duplication if x is not None]

section_names = ["SKU Share By Brand","SKU Share by Manuf" if RunManuf else None,"Cumulative Product Shares","Manuf Cumulative Product Share" if len(client_brands) >0 else None,"Top 20 cumulative share","SKU Productivity Analysis with TM%" if len(client_brands)>0 else None, "SKU Productivity Analysis with WD" if len(client_brands)>0  else None,
                 #"Top 50% cumulative share",##TOP%CUM
                 "Sectors Fair Share" if len(sectors) > 0 else None, "Segments Fair Share" if len(segments) > 0 else None, "SubCategory Fair Share" if len(subcategories) > 0 else None, "SubSegment Fair Share" if len(subsegments) > 0 else None,"SKU Share By Brand no client prio" if Runwithoutclientorder else None ]
section_names = [x for x in section_names if x is not None]

path = os.getcwd() + '//Assortment base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'


length = len(duplication)
for i in reversed(range(length)):
    if duplication[i]==0:
        del duplication[i]
        del index[i]
        del section_names[i]


In [384]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)


8
[0, 1, 3, 4, 5, 6, 7, 7]
8
[240, 240, 192, 240, 94, 94, 32, 32]
8
['SKU Share By Brand', 'Cumulative Product Shares', 'Brand Cumulative Product Share', 'Top 20 cumulative share', 'SKU Productivity Analysis with TM%', 'SKU Productivity Analysis with WD', 'Sectors Fair Share', 'Segments Fair Share']


In [385]:
sum(duplication)

1164

In [386]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [387]:
prs = Presentation(new_pre)

### Slide 1

In [388]:
posItr = 0
ind=0
SkuShareByBrand(prs,assortmentBrandSorted,duplication[ind],BrandOrTopB,position=posItr)
posItr +=len(assortmentBrandSorted)
ind+=1

In [389]:
print(posItr,ind)

240 1


In [390]:
if RunManuf:
    SkuShareByBrand(prs,assortmentManufSorted,duplication[ind],ManufOrTopC,position=posItr)
    posItr += len(assortmentManufSorted)
    ind+=1

In [391]:
print(posItr,ind)

240 1


### Slide 2

In [392]:
CumulativeProductShare(prodORitemfields,prs,assortmentBrand,duplication[ind],position=posItr)
posItr+= len(assortmentBrand)
ind+=1

In [393]:
print(posItr,ind)

480 2


### Slide 3

In [394]:
# cumulativeTop50(prodORitem,prs, cumulativeShareTop50,duplication[ind],position=posItr)
# posItr +=len(cumulativeShareTop50)
# ind+=1

In [395]:
print(posItr,ind)

480 2


### Slide 4

In [396]:
if len(client_brands)>0:
    brandCumulativeProductShare(prodORitem,prodORitemfields,prs,assortmentClientManufAppend,duplication[ind],position=posItr)
    posItr +=len(assortmentClientManufAppend)
    ind+=1

In [397]:
print(posItr,ind)

672 3


### Slide 5

In [398]:
top20CumulativeShare(prodORitem,prodORitemfields,prs,assortmentBrand,duplication[ind],client_brands, position=posItr)
posItr+= len(assortmentBrand)
ind+=1

In [399]:
print(posItr,ind)

912 4


### Slide 6

In [400]:
if len(client_brands)>0 and len(assortmentClientBrand.keys())>0:
    SKUProductivityAnalysis(prodORitem,prodORitemfields,prs,assortmentClientBrand,duplication[ind],position=posItr)
    posItr += len(assortmentClientBrand)
    ind+=1

In [401]:
print(posItr,ind)

1006 5


### Slide 7

In [402]:
if len(client_brands)>0 and len(assortmentClientBrand.keys())>0:
    SKUWithWD(prodORitem,prodORitemfields,prs,assortmentClientBrand,duplication[ind],position=posItr)
    posItr += len(assortmentClientBrand)
    ind+=1

In [403]:
print(posItr,ind)

1100 6


### Slide 8

In [404]:

if len(sectors)>0:
    fair_share(final_Sector,prs ,scope ='Sector', position=posItr,category = categories[0])
    posItr += len(final_Sector)
    ind+=1

In [405]:
if len(segments)>0:
    fair_share(final_Segment,prs ,scope='Segment', position=posItr,category =categories[0])
    posItr += len(final_Segment)
    ind+=1
    

In [406]:
if len(subcategories)>0:
    fair_share(final_SubCategory,prs ,scope='SubCategory', position=posItr,category =categories[0])
    posItr += len(final_SubCategory)
    ind+=1
    

In [407]:
if len(subsegments)>0:
    fair_share(final_Subsegment,prs ,scope='SubSegment', position=posItr,category =categories[0])
    posItr += len(final_Subsegment)
    ind+=1

### Slide 1 without forcing client brand

In [408]:
print(posItr,ind)

1164 8


In [409]:
if Runwithoutclientorder: SkuShareByBrand(prs,assortmentBrandNOTSorted,duplication[ind],BrandOrTopB,position=posItr)

In [410]:
outputPath=os.getcwd() + f"\\Mix and assortment output_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [411]:
final=os.getcwd() + f"\\Mix and assortment output_{datetime.today().strftime("%d-%m")}.pptx"
pptx_path = outputPath  # Replace with the actual path to your PPTX file
output_pptx_path=final
open_chart_data_in_excel(pptx_path,output_pptx_path)


Slide 913: Opened Excel workbook: Book1
Slide 914: Opened Excel workbook: Book1
Slide 915: Opened Excel workbook: Book1
Slide 916: Opened Excel workbook: Book1
Slide 917: Opened Excel workbook: Book1
Slide 918: Opened Excel workbook: Book1
Slide 919: Opened Excel workbook: Book1
Slide 920: Opened Excel workbook: Book1
Slide 921: Opened Excel workbook: Book1
Slide 922: Opened Excel workbook: Book1
Slide 923: Opened Excel workbook: Book1
Slide 924: Opened Excel workbook: Book1
Slide 925: Opened Excel workbook: Book1
Slide 926: Opened Excel workbook: Book1
Slide 927: Opened Excel workbook: Book1
Slide 928: Opened Excel workbook: Book1
Slide 929: Opened Excel workbook: Book1
Slide 930: Opened Excel workbook: Book1
Slide 931: Opened Excel workbook: Book1
Slide 932: Opened Excel workbook: Book1
Slide 933: Opened Excel workbook: Book1
Slide 934: Opened Excel workbook: Book1
Slide 935: Opened Excel workbook: Book1
Slide 936: Opened Excel workbook: Book1
Slide 937: Opened Excel workbook: Book1
